# Static Load Calculations for Snow Slabs

This notebook demonstrates methods for calculating the static load (weight per unit area) of layered snow slabs and their force components on slopes using the `snowpyt_mechparams` package.

## Overview

The static load calculations include:
- **Gravitational load**: Total weight per unit area (vertical force)
- **Shear load**: Force component parallel to the slope surface
- **Normal load**: Force component perpendicular to the slope surface

These calculations are essential for avalanche hazard assessment and snow stability analysis.


In [10]:
# Import Libraries
import os
import sys

import pandas as pd

# Import snowpylot for CAAML parsing
from snowpylot import caaml_parser

# Add the src directory to the path to import snowpyt_mechparams
sys.path.append('../src')
from snowpilot_utils import convert_grain_form
from snowpyt_mechparams import density
from snowpyt_mechparams.data_structures import Layer, Slab
from snowpyt_mechparams.static_load import calculate_static_load


## Parse Snowpit Files

Parse CAAML snowpit files to extract real layer data:


In [ ]:
all_pits = []
failed_files = []

folder_path = 'data'

xml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')]

for file in xml_files:
    try:
        file_path = os.path.join(folder_path, file)
        pit = caaml_parser(file_path)
        all_pits.append(pit)
    except Exception as e:
        failed_files.append((file, str(e)))
        print(f"Warning: Failed to parse {file}: {e}")

print(f"Successfully parsed {len(all_pits)} files")
print(f"Failed to parse {len(failed_files)} files")


Successfully parsed 50278 files
Failed to parse 0 files


## Construct Slab

In [21]:
# UPDATED VERSION - Only create slab if ALL layers above layer of concern have successful density calculations
# Now returns dictionary with pit_id -> slab mapping

slabs = {}  # Changed to dictionary to store pit_id -> slab mapping
skipped_pits = 0
pits_w_angle_and_layer_of_concern = 0
density_calc_failures = 0

for pit in all_pits:
    pit_id = pit.core_info.pit_id
    layers = pit.snow_profile.layers

    # Check if pit has slope angle and contains a layer of concern
    has_slope_angle = pit.core_info.location.slope_angle is not None
    has_layer_of_concern = any(getattr(layer, 'layer_of_concern', False) for layer in layers)

    if has_slope_angle and has_layer_of_concern:
        pits_w_angle_and_layer_of_concern += 1
        slab_layers = []
        all_layers_valid = True  # Track if all layers are successfully processed

        for layer in layers:
            if layer.layer_of_concern:
                break # Stop at layer of concern

            layer_hardness = layer.hardness
            layer_geldsetzer_grain_form = convert_grain_form(layer.grain_form_primary)

            # Reject entire pit if any layer has missing required data
            if layer_hardness is None or layer_geldsetzer_grain_form is None:
                all_layers_valid = False
                break # Stop processing this pit

            try:
                layer_density = density.calculate_density(
                    method='geldsetzer',
                    hand_hardness=layer_hardness,
                    grain_form=layer_geldsetzer_grain_form
                )
                # Create layer object if density calculation succeeded
                layer_obj = Layer(thickness=layer.thickness, density=layer_density)
                slab_layers.append(layer_obj)
            except Exception as e:
                density_calc_failures += 1
                all_layers_valid = False
                break # Reject entire pit if any density calculation fails

        # Only create slab if ALL layers above layer of concern were successfully processed
        if all_layers_valid and slab_layers:
            slab = Slab(layers=slab_layers, angle=pit.core_info.location.slope_angle)
            slabs[pit_id] = slab  # Store slab with pit_id as key
        else:
            skipped_pits += 1

print(f"Found {len(slabs)} valid slabs from {pits_w_angle_and_layer_of_concern} candidate pits")
print(f"Skipped {skipped_pits} pits (rejected if any layer above layer of concern failed)")
print(f"Total density calculation failures: {density_calc_failures}")


Found 10482 valid slabs from 33279 candidate pits
Skipped 22797 pits (rejected if any layer above layer of concern failed)
Total density calculation failures: 9992


## Static Load Calculations

